In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# @markdown Loading Dependencies
import os
from google.colab import files
!pip install -q kaggle
import shutil
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from zipfile import ZipFile
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from keras.models import Sequential, load_model
from keras.layers import (GRU, LSTM, Bidirectional, Dense, Dropout, Flatten,
                          SimpleRNN)
from keras.optimizers import (SGD, RMSprop, Adagrad,
                              Adadelta, Adam, Adamax, Nadam)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_squared_error,mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials,space_eval
from keras.models import load_model,save_model

try:
  !mkdir models
except:
  pass

In [ ]:
# # @markdown importing dataset from kaggle
# src = os.path.join('/content/gdrive/My Drive/covid-19', 'kaggle.json')
# dst = os.path.join('/content', 'kaggle.json')
# shutil.copy(src, dst)
# !mkdir ~/.kaggle
# !cp /content/kaggle.json ~/.kaggle/kaggle.json
# !kaggle datasets download -d sudalairajkumar/covid19-in-india


# with ZipFile('/content/covid19-in-india.zip', 'r') as zip:
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

data = pd.read_csv("/content/covid_19_india.csv")

In [ ]:
data.tail(3)

,Date,Name of State / UT,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
4689,2020-08-06,Uttar Pradesh,26.8467,80.9462,104388.0,1857,60558.0,4078,0,3287
4690,2020-08-06,Uttarakhand,30.0668,79.0193,8254.0,98,5233.0,246,0,386
4691,2020-08-06,West Bengal,22.9868,87.8550,83800.0,1846,58962.0,2816,0,2078


In [ ]:
# @markdown Dataset Preparation
Dates = []
for i in data['Date']:
  if i not in Dates:
    Dates.append(i)

Date=[]
Confirmed_cases = []
for i in Dates:
  df = data[data['Date']==i]
  Date.append(i)
  Confirmed_cases.append(df['Total Confirmed cases'].sum())

data_new = {'Date' : Date, 'Confirmed Cases': Confirmed_cases}
df_train = pd.DataFrame(data_new)

# Print the output.
#df_train=df_train.set_index('Date')

df_train.tail(2)

,Date,Confirmed Cases
184,2020-08-05,1908254.0
185,2020-08-06,1964536.0


In [ ]:
df_train =df_train.head(len(df_train.index)-1)
df_train.tail()

,Date,Confirmed Cases
180,2020-08-01,1695988.0
181,2020-08-02,1750723.0
182,2020-08-03,1803695.0
183,2020-08-04,1855745.0
184,2020-08-05,1908254.0


In [ ]:
# @markdown Visualization 1

fig = go.Figure()
trace = fig.add_trace(go.Scatter(x=df_train['Date'], y=df_train['Confirmed Cases'], name='Confirmed Cases in India', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=df_train['Confirmed Cases']))
fig.show()

In [ ]:
# @markdown Visualization 2
fig2 = px.bar(df_train, x="Date", y="Confirmed Cases", color='Confirmed Cases', orientation='v', height=600,
             title='Confirmed Cases in India', color_discrete_sequence = px.colors.cyclical.Twilight)
fig2.show()

In [ ]:
# @markdown Data Preprocessing
val = df_train['Confirmed Cases'].values
val_index = df_train['Date'].values
val = val.astype('float32')
#print(val_index)
#print(val)
val = np.reshape(val, (-1, 1))
#print(val)
scaler = MinMaxScaler(feature_range=(0, 1))
val = scaler.fit_transform(val)
#print(val)

In [ ]:
# @markdown Preparation of training and testing data
train_size = int(len(val) * 0.75)
print(train_size)
test_size = len(val) - train_size
print(test_size)
train, test = val[0:train_size,:], val[train_size:len(val),:]
index_train, index_test = val_index[0:train_size], val_index[train_size:len(val_index)]

def create_dataset(dataset, look_back, index_dataset):
    X, Y ,index_X,index_Y= [], [], [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        #print('x is : {}'.format(X))
        Y.append(dataset[i + look_back, 0])
        index_Y.append(index_dataset[i + look_back])
        #print('y is : {}'.format(Y))
        c = index_dataset[i:(i+look_back)]
        index_X.append(c)
    return np.array(X), np.array(Y),np.array(index_X),np.array(index_Y)

look_back = 10

X_train, Y_train, index_X_train, index_Y_train = create_dataset(train, look_back, index_train)
X_test, Y_test, index_X_test, index_Y_test = create_dataset(test, look_back, index_test)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

138
47


In [ ]:
len(X_train)

127

In [ ]:
len(X_test)

36

In [ ]:
model= Sequential()
input_shape=(X_train.shape[1], X_train.shape[2])

In [ ]:
# @markdown RNN Model Training

model.add(SimpleRNN(
                200,
                return_sequences=True,
                activation='relu',
                input_shape=input_shape
                ))

model.add(Dropout(0.1))
model.add(SimpleRNN(
                    300,
                    return_sequences=True,
                    activation='tanh'
                    )
                )
model.add(Dropout(0.1))
model.add(SimpleRNN(
                100,
                return_sequences=True,
                activation='relu'
                )
            )
model.add(Flatten())


model.add(Dense(1, activation='linear',))

model.compile(loss='mean_squared_error', optimizer='Adam')
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]
history = model.fit(X_train, Y_train,
                    epochs=80,
                    batch_size=50,
                    callbacks=callbacks,
                    verbose=0,
                    shuffle=False)



In [ ]:
# model = load_model('/content/models/modelwithloss557.h5')


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
# @markdown Performing predictions on training and testing
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict = scaler.inverse_transform(train_predict)
Y_train_ = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test_ = scaler.inverse_transform([Y_test])
print('Train Mean Absolute Error:', mean_absolute_error(Y_train_[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train_[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test_[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test_[0], test_predict[:,0])))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Train Mean Absolute Error: 2045.5339216769792
Train Root Mean Squared Error: 2842.587037483532
Test Mean Absolute Error: 45416.727397960014
Test Root Mean Squared Error: 53969.137437907186


In [ ]:
accuracy = 100 -(mean_absolute_percentage_error(Y_test_[0], test_predict[:,0]))
accuracy

95.4443867454152

In [ ]:
# @markdown Post Processing
train_pred = list(train_predict.flatten())
y_train = list(Y_train_.flatten())
y_test = list(Y_test_.flatten())
test_pred = list(test_predict.flatten())

for i in range(len(y_train)):
  train_pred[i] = int(round(train_pred[i]))
  y_train[i] = int(round(y_train[i]))
for i in range(len(y_test)):
  test_pred[i] = int(round(test_pred[i]))
  y_test[i] = int(round(y_test[i]))

dict_train = {'Date': list(index_Y_train), 'Actual': y_train,'Predicted' :train_pred}
dict_test = {'Date': list(index_Y_test), 'Actual': y_test ,'Predicted' :test_pred}

train_df = pd.DataFrame(dict_train)
test_df = pd.DataFrame(dict_test)

test_df.tail(5)

,Date,Actual,Predicted
31,2020-07-31,1638870,1554290
32,2020-08-01,1695988,1608422
33,2020-08-02,1750723,1668414
34,2020-08-03,1803695,1736976
35,2020-08-04,1855745,1799434


In [ ]:
# @markdown For Training
fig = go.Figure()
trace1 = fig.add_trace(go.Scatter(x=train_df['Date'], y=train_df['Actual'], name='Actual', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=train_df['Actual']))
trace2 = fig.add_trace(go.Scatter(x=train_df['Date'], y=train_df['Predicted'], name='Predicted', mode='lines+markers',
                         line_color='#49a7c7',line_width=1.5, hovertext=train_df['Predicted']))
fig.show()

In [ ]:
# @markdown For Testing

fig2 = go.Figure()
trace1 = fig2.add_trace(go.Scatter(x=test_df['Date'], y=test_df['Actual'], name='Actual', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=test_df['Actual']))
trace2 = fig2.add_trace(go.Scatter(x=test_df['Date'], y=test_df['Predicted'], name='Predicted', mode='lines+markers',
                         line_color='#49a7c7',line_width=1.5, hovertext=test_df['Predicted']))
fig2.show()

# Performing Hyperparameter Tuning
Bayesian optimization using HyperOpt library(Tpe algorithm)

In [ ]:
# @markdown Objective Function
def createModel(space):
  model= Sequential()
  input_shape=(X_train.shape[1], X_train.shape[2])
  print(input_shape)
  model.add(SimpleRNN(space['neuron0'],
                      return_sequences=True,
                      activation = space['activation0'],
                      input_shape=input_shape))
  model.add(Dropout(space['dropout0']))
  model.add(SimpleRNN(space['neuron1'],return_sequences=True,activation = space['activation1'],input_shape=input_shape))
  model.add(Dropout(space['dropout1']))
  model.add(SimpleRNN(space['neuron2'],return_sequences=True,activation = space['activation2'],input_shape=input_shape))
  model.add(Flatten())
  model.add(Dense(1, activation='linear',))
  model.compile(loss='mean_squared_error', optimizer='Adam')
  callbacks = [ModelCheckpoint(monitor='val_loss', filepath='/content/mymodel.keras', save_best_only=False,save_weights_only=False)]
  history = model.fit(X_train, Y_train,epochs=space['epochs'],batch_size=space['batch_size'],callbacks=callbacks,verbose=1,shuffle=False)
  test_predict = model.predict(X_test)
  test_predict = scaler.inverse_transform(test_predict)
  Y_test_ = scaler.inverse_transform([Y_test])
  mae = mean_absolute_error(Y_test_[0], test_predict[:,0])
  mse = np.sqrt(mean_squared_error(Y_test_[0], test_predict[:,0]))
  round_mae = int(round(mae))
  file_path = '/content/models/modelwithloss' + str(round_mae) + '.keras'
  model.save(file_path)
  return {'loss': mae, 'status': STATUS_OK}


In [ ]:
# @markdown Definition of Space
space = {
    'neuron0' : hp.choice('neurons0',[50,100,150,200,250,300]),
    'neuron1' : hp.choice('neurons1',[50,100,150,200,250,300]),
    'neuron2' : hp.choice('neurons2',[50,100,150,200,250,300]),
    'dropout0' : hp.choice('dropout0',[0.1,0.2,0.3,0.4,0.5]),
    'dropout1' : hp.choice('dropout1',[0.1,0.2,0.3,0.4,0.5]),
    'epochs' : hp.choice('epochs',[20,50,80,100]),
    'batch_size' : hp.choice('batch_size',[20,50,80,100]),
    'activation0' : hp.choice('activation0',['tanh','relu']),
    'activation2' : hp.choice('activation1',['tanh','relu']),
    'activation1' : hp.choice('activation2',['tanh','relu'])
}

In [ ]:
# @markdown Performing Hyper-parameter tuning
trials=Trials()
tpe_best=fmin(createModel,
              space=space,
              algo=tpe.suggest,
              max_evals = 25,
              trials=trials,
              verbose=2)
loss=trials.best_trial['result']['loss']
best_params=space_eval(space,tpe_best)
print('Best Loss: {}'.format(loss))
print('Best Parameters by index: {}'.format(tpe_best))
print('Best Parameters: {}'.format(best_params))


(1, 10)
Epoch 1/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step - loss: 9.2369e-13
5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.9100e-05
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 1.7154e-04

Epoch 2/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.0037
5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0032 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0031

Epoch 3/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 1.1212e-07
4/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.5765e-04 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0012    

Epoch 4/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 4.9427e-04
5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.6188e-04 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.6941e-04

Epoch 5/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 4.9238e-04
5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 6.5315e-04 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 7.2232e-04

Epoch 6/50

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0013
5/7 ━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




1/7 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - loss: 5.3566e-13
5/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.8223e-05
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 1.3343e-04

Epoch 2/80

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.0011
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0014  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0016

Epoch 3/80

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 7.3726e-06
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.0100e-04 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 5.0431e-04

Epoch 4/80

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 7.4813e-06
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 9.9339e-05 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.5990e-04

Epoch 5/80

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 1.9678e-04
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.1565e-04 

Epoch 6/80

1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 3.5778e-04
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.2518e-04 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
  4%|▍         | 1/25 [00:34<05:59, 14.98s/trial, best loss: 371376.6236744013]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 376ms/step

(1, 10)
  8%|▊         | 2/25 [00:36<07:05, 18.52s/trial, best loss: 177010.7712438458]

...

Epoch 1/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 2.5666e-08
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0014  

Epoch 2/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 5.1514e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 6.3734e-04 

Epoch 3/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0012
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 9.9419e-04

Epoch 4/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 7.6415e-06
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 1.2737e-04 

Epoch 5/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 4.6826e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 6.1053e-04

Epoch 6/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 1.8958e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.0483e-04

Epoch 7/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 4.3328e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 1.5820e-04

Epoch 8/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 1.9465e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




1/3 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 2.8310e-08
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0014  

Epoch 2/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 5.0660e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 8.7055e-04

Epoch 3/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - loss: 9.0854e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 7.8517e-04 

Epoch 4/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.6803e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.8748e-04

Epoch 5/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 5.1901e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 6.7750e-04 

Epoch 6/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 1.2108e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 1.3625e-04

Epoch 7/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 8.1297e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 2.5561e-04

Epoch 8/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 2.3258e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - loss: 1.7328e-08
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.0013  

Epoch 2/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 3.9906e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 7.4998e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 8.7804e-04

Epoch 3/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 6.9081e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 6.0920e-04 

Epoch 4/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 2.6994e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.2275e-04 

Epoch 5/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 4.8903e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 6.7266e-04

Epoch 6/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - loss: 1.3218e-04
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.4122e-04 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 1.4453e-04

Epoch 7/100

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 8.2381e-05
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 2.1092e-04

E

In [ ]:
# @markdown Future 7 days Predictions
import numpy as np
from sklearn.preprocessing import MinMaxScaler
model=load_model("/content/models/modelwithloss28107.keras")
lastdate = df_train['Date'][len(df_train.index)-1]
lastdate=lastdate.split('-')[2]
date=int(lastdate) + 1
df_test = df_train.tail(look_back)

for i in range(15):
  val = df_test['Confirmed Cases'].values
  val_index = df_test['Date'].values
  val = val.astype('float32')
  val = np.reshape(val, (-1, 1))
  scaler = MinMaxScaler(feature_range=(0, 1))
  val = scaler.fit_transform(val)
  X_future_test= []
  X_future_test.append(val[len(val)-look_back:])
  X_future_test =np.array(X_future_test)
  X_future_test = np.reshape(X_future_test, (X_future_test.shape[0], 1, X_future_test.shape[1]))
  future_predict = model.predict(X_future_test)
  future_predict = scaler.inverse_transform(future_predict)
  future_pred = list(future_predict.flatten())
  next_date = str(date) + '/05/20'
  date = date+1
  future_pred[0] = int(round(future_pred[0]))
  dict_train = {'Date': [next_date] , 'Confirmed Cases' :future_pred}
  future_df = pd.DataFrame(dict_train)
  # df_test = df_test.append(future_df,ignore_index = True)
  df_test = pd.concat([df_test, future_df], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [ ]:
df_test.head(5)

,Date,Confirmed Cases
0,2020-07-27,1376735.0
1,2020-07-28,1483156.0
2,2020-07-29,1531669.0
3,2020-07-30,1583792.0
4,2020-07-31,1638870.0


In [ ]:
future_pred

[2014591]